In [294]:
import sys
import os
import numpy as np
import pandas as pd
from FCLayer import FcLayer
from ACLayer import AcLayer
from Network import Network
from activation_functions import sigmoid, relu, sigmoid_prime, tanh, tanh_prime,relu_prime
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [272]:
# x_train = np.array([[[0, 0]], [[0, 1]], [[1, 0]], [[1, 1]]])
# # x_train = np.array([[[10000, 1111]], [[111110, 5000]], [[100, 220]]])
# # y_train = np.array([[[0]], [[1]], [[1]], [[0]]])
# y_train = np.array([[[0]], [[1]], [[1]], [[0]]])
'''
df_vote = pd.read_csv("CongressionalVotingID.shuf.lrn.csv", index_col="ID")
df_vote = df_vote.applymap(lambda x: 1 if x == "y" else 0 if x == "n" else x)
df_vote = df_vote.replace("democrat", 1)
df_vote = df_vote.replace("republican", 0)
df_vote = df_vote.replace("unknown", 0)
df_vote.head()

df_vote.shape

y_train = df_vote.iloc[:, 0]
X_train = df_vote.iloc[:, 1:]
X_train.shape
y_test = pd.read_csv("CongressionalVotingID.shuf.sol.ex.csv", index_col="ID")
y_test = y_test.replace("democrat", 1)
y_test = y_test.replace("republican", 0)
y_test
# Create x_val
df_test = pd.read_csv("CongressionalVotingID.shuf.tes.csv")
df_test = df_test.applymap(lambda x: 1 if x == "y" else 0 if x == "n" else x)
df_test = df_test.replace("unknown", 0)
X_test = df_test.iloc[:, 1:]
X_test
'''

# x_train = np.array([[[0, 0]], [[0, 1]], [[1, 0]], [[1, 1]]])
# x_train = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
# y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

'''
x_train = np.expand_dims(X_train.to_numpy(), axis=1)
y_train = np.expand_dims(y_train.to_numpy(), axis=1)
x_test = np.expand_dims(X_test.to_numpy(), axis=1)
y_test = np.expand_dims(y_test.to_numpy(), axis=1)
'''

'\nx_train = np.expand_dims(X_train.to_numpy(), axis=1)\ny_train = np.expand_dims(y_train.to_numpy(), axis=1)\nx_test = np.expand_dims(X_test.to_numpy(), axis=1)\ny_test = np.expand_dims(y_test.to_numpy(), axis=1)\n'

In [273]:
# Set random seed for reproducibility
np.random.seed(42)

df_vote = pd.read_csv("CongressionalVotingID.shuf.lrn.csv", index_col="ID")
df_vote = df_vote.applymap(lambda x: 1 if x == "y" else 0 if x == "n" else x)
df_vote = df_vote.replace("democrat", 1)
df_vote = df_vote.replace("republican", 0)
df_vote = df_vote.replace("unknown", 0)



In [274]:
x_train, x_test, y_train, y_test = train_test_split(df_vote.iloc[:,1:], df_vote.iloc[:, 0],test_size=0.4, random_state=11)

In [275]:
x_train = np.expand_dims(x_train.to_numpy(), axis=1)
y_train = np.expand_dims(y_train.to_numpy(), axis=1)
x_test = np.expand_dims(x_test.to_numpy(), axis=1)
y_test = np.expand_dims(y_test.to_numpy(), axis=1)

In [276]:
np.unique(y_test)

array([0, 1])

In [277]:

learning_rate = 0.1


network1 = Network(learning_rate)

# fc1 = FcLayer(2, 3)
# ac1 = AcLayer(tanh, tanh_prime)
# fc2 = FcLayer(3, 1)
# ac2 = AcLayer(tanh, tanh_prime)
# network1.add(fc1)
# network1.add(ac1)
# network1.add(fc2)
# network1.add(ac2)

input_values = 16
output_values = 1
intermediate_values = 3
layers = 3
epochs = 1000

for j in range(layers):
    i = intermediate_values
    o = intermediate_values

    if not j:
        i = input_values
    elif j + 1 == layers:
        o = output_values

    fc = FcLayer(i, o)
    ac = AcLayer(tanh, tanh_prime)
    network1.add(fc)
    network1.add(ac)

network1.train(x_train, y_train, epochs=epochs)

out = network1.predict(x_train)
#print(out)


# fc1.forward_propagation(input)
# fc2.forward_propagation(fc1.output)
# print(fc2.output)

# x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
# y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# # network
# net = Network()
# net.add(FCLayer(2, 3))
# net.add(ActivationLayer(tanh, tanh_prime))
# net.add(FCLayer(3, 1))
# net.add(ActivationLayer(tanh, tanh_prime))


# train
# net.use(mse, mse_prime)
# net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# # test
# out = net.predict(x_train)
# print(out)


Predict function works
[array([[0.99859773]]), array([[0.00088103]]), array([[0.99860138]]), array([[0.99860308]]), array([[0.99686684]]), array([[0.00085926]]), array([[0.99859923]]), array([[0.99859925]]), array([[0.99859908]]), array([[-0.00288051]]), array([[0.00093866]]), array([[0.00099955]]), array([[0.99859063]]), array([[0.99860288]]), array([[0.99858662]]), array([[0.00100804]]), array([[0.99858215]]), array([[0.00099955]]), array([[0.99859783]]), array([[-0.00031842]]), array([[-0.00092704]]), array([[0.00103764]]), array([[0.99854944]]), array([[0.99859763]]), array([[0.99522904]]), array([[0.00100804]]), array([[0.00061491]]), array([[0.99860163]]), array([[0.99860342]]), array([[0.00088103]]), array([[0.9986025]]), array([[0.00198315]]), array([[0.99860145]]), array([[0.99854816]]), array([[0.99860293]]), array([[0.99622269]]), array([[0.99860328]]), array([[0.99849602]]), array([[0.99860334]]), array([[0.00012857]]), array([[0.99859923]]), array([[-0.00240492]]), array([

In [278]:
y_pred=network1.predict(x_test)
y_pred=np.where(np.array(y_pred) > 0.5, 1, 0)
np.unique(y_pred)
#y_test==y_pred

Predict function works
[array([[-0.00343621]]), array([[0.00095814]]), array([[0.99821491]]), array([[0.00092663]]), array([[0.99859933]]), array([[0.00088103]]), array([[-0.00587643]]), array([[0.00091049]]), array([[0.99860172]]), array([[0.00095814]]), array([[0.99856898]]), array([[-0.0047495]]), array([[-0.00110698]]), array([[0.05871406]]), array([[0.99860306]]), array([[0.00103799]]), array([[0.00103815]]), array([[0.99860076]]), array([[0.99860336]]), array([[0.99857781]]), array([[0.99859607]]), array([[0.99860297]]), array([[0.0250302]]), array([[0.00095814]]), array([[0.99859607]]), array([[0.89179223]]), array([[0.99858215]]), array([[0.00100189]]), array([[0.99860303]]), array([[-0.06451908]]), array([[0.97143108]]), array([[0.99858215]]), array([[-0.00273613]]), array([[0.00103257]]), array([[0.99860293]]), array([[0.99859908]]), array([[0.0010361]]), array([[0.99859956]]), array([[0.99860277]]), array([[0.99859907]]), array([[0.00082206]]), array([[0.00103799]]), array([

array([0, 1])

***Classification report for Self Neural Network***

In [279]:
print(classification_report(y_pred=np.squeeze(y_pred),y_true=np.squeeze(y_test)))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96        37
           1       1.00      0.94      0.97        51

    accuracy                           0.97        88
   macro avg       0.96      0.97      0.97        88
weighted avg       0.97      0.97      0.97        88



***Tensor Flow***

In [280]:
import tensorflow as tf

In [281]:
def train_model(x_train,y_train, num_nodes, dropout_prob,lr, batch_size, epochs):
    nn_model=tf.keras.Sequential([tf.keras.layers.Dense(num_nodes, activation='relu', input_shape=(16,)),
                              tf.keras.layers.Dropout(dropout_prob),
                             tf.keras.layers.Dense(num_nodes, activation='relu'),
                             tf.keras.layers.Dropout(dropout_prob),
                             tf.keras.layers.Dense(1, activation='sigmoid')])

    nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss="binary_crossentropy",metrics=['accuracy'])

    #x_train=x_train.astype(float)

    history = nn_model.fit(x_train,y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
    return nn_model, history

In [282]:
nn_model, history= train_model(np.squeeze(x_train),np.squeeze(y_train),16 , 0,0.1, 32, 1000)

In [283]:
y_pred=nn_model.predict(np.squeeze(x_test))
y_pred=(y_pred>0.5).astype(int).reshape(-1)
y_pred


print(classification_report(y_pred,np.squeeze(y_test)))

3/3 [==============================] - 0s 914us/step
              precision    recall  f1-score   support

           0       0.97      0.95      0.96        38
           1       0.96      0.98      0.97        50

    accuracy                           0.97        88
   macro avg       0.97      0.96      0.97        88
weighted avg       0.97      0.97      0.97        88



***Spam Data Self Neural Network***

In [295]:
from sklearn.preprocessing import StandardScaler
data=pd.read_csv('spam_final_df.data')
data.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [296]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=26)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [297]:
X_test=np.expand_dims(X_test,axis=1)
X_train=np.expand_dims(X_train,axis=1)
y_train=np.expand_dims(y_train,axis=1)
y_test=np.expand_dims(y_test,axis=1)
network1 = Network(learning_rate)


In [299]:
fc1 = FcLayer(57,50)
ac1 = AcLayer(relu, relu_prime)
fc2 = FcLayer(50, 40)
ac2 = AcLayer(relu, relu_prime)
fc3 = FcLayer(40, 10)
ac3 = AcLayer(relu, relu_prime)
fc4 = FcLayer(30, 20)
ac4 = AcLayer(relu, relu_prime)
fc5 = FcLayer(20, 10)
ac5 = AcLayer(sigmoid, sigmoid_prime)
fc10 = FcLayer(10, 1)
ac10= AcLayer(relu, relu_prime)
network1.add(fc1)
network1.add(ac1)
network1.add(fc2)
network1.add(ac2)

network1.add(fc3)
network1.add(ac3)
# network1.add(fc4)
# network1.add(ac4)
# network1.add(fc5)
# network1.add(ac5)

# network1.add(fc6)
# network1.add(ac6)
# network1.add(fc7)
# network1.add(ac7)
# network1.add(fc8)
# network1.add(ac8)

# network1.add(fc9)
# network1.add(ac9)
network1.add(fc10)
network1.add(ac10)

In [300]:
epochs=1000
network1.train(X_train, y_train, epochs=epochs)

y_pred = network1.predict(X_test)
print("y_pred unique")
print(np.unique(y_pred))

ValueError: shapes (1,1) and (57,50) not aligned: 1 (dim 1) != 57 (dim 0)